In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spikeinterface.full as si
import spikeinterface.sortingcomponents.peak_detection as pkd
import numpy as np
import time

/home/aurelien/Documents/Rust/spikeinterface_rust/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": True, "chunk_duration":"1s"}

In [4]:
rec, sorting = si.generate_ground_truth_recording(durations=[60],num_channels=128,sampling_frequency=25000.0,num_units= 100, seed=42)
rec = rec.save_to_memory(job_kwargs=job_kwargs)
_ = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

noise_level (workers: 20 threads): 100%|██████████| 20/20 [00:00<00:00, 189.95it/s]


Compute noise levels time: 0.107 s


detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 194.89it/s]

Detect peaks 'locally_exclusive' total time: 0.418 s


In [5]:
peaks_numba = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)
peaks_rust = pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

print(len(peaks_numba))
print(len(peaks_rust))

Compute noise levels time: 0.000 s


detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 737.06it/s]


Detect peaks 'locally_exclusive' total time: 0.084 s
Compute noise levels time: 0.000 s


detect peaks (locally_exclusive) (workers: 24 threads): 100%|██████████| 60/60 [00:00<00:00, 1052.37it/s]

Detect peaks 'locally_exclusive' total time: 0.059 s
77651
77341


In [6]:
print(peaks_numba[-10:-1])
print(peaks_rust[-10:-1])

[(1499805, 107,  -74.8041153 , 0) (1499809,  12,  -42.23897552, 0)
 (1499817,  88, -120.50748444, 0) (1499826,  26,  -84.51519775, 0)
 (1499847,   2,  -81.30005646, 0) (1499855, 108,  -55.4861145 , 0)
 (1499883,  81, -167.32948303, 0) (1499889,  56,  -39.03798676, 0)
 (1499987,  90,  -70.16048431, 0)]
[(1499805, 107,  -74.8041153 , 0) (1499809,  12,  -42.23897552, 0)
 (1499817,  88, -120.50748444, 0) (1499826,  26,  -84.51519775, 0)
 (1499847,   2,  -81.30005646, 0) (1499855, 108,  -55.4861145 , 0)
 (1499883,  81, -167.32948303, 0) (1499889,  56,  -39.03798676, 0)
 (1499987,  90,  -70.16048431, 0)]


In [7]:
np.setdiff1d(peaks_numba, peaks_rust)

array([(   6526,  88, -118.95065308, 0), (   7011,   9, -131.99391174, 0),
       (  12800, 111,  -65.21321869, 0), (  17645,  45,  -75.91956329, 0),
       (  18163,  99, -166.37471008, 0), (  23436,  47,  -72.32080078, 0),
       (  24471,  25,  -85.23049927, 0), (  26988,   7, -202.05363464, 0),
       (  32818,  47,  -71.81715393, 0), (  35723,  47,  -71.72771454, 0),
       (  42230,  55,  -38.35082245, 0), (  43703, 110,  -90.93703461, 0),
       (  44220,  18, -104.37729645, 0), (  45281,  56,  -31.55821228, 0),
       (  59211,  83,  -84.04824066, 0), (  60150, 119, -150.43037415, 0),
       (  61576, 116, -216.44604492, 0), (  63252,   6,  -50.01287842, 0),
       (  79659,   9, -199.4281311 , 0), (  81667,  30, -213.02798462, 0),
       (  82540,  55,  -99.52503967, 0), (  91044,  41,  -61.47731781, 0),
       (  92924, 104, -127.52288818, 0), (  99925,  27, -141.99012756, 0),
       ( 102907,  60, -152.20393372, 0), ( 105604, 113, -113.89753723, 0),
       ( 106441,  31, -17

In [ ]:
for i in range(1,4):
    job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
    print("Chunk duration :" + str(500*i) +" ms")
    %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

Chunk duration :100ms
249 ms ± 6.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Chunk duration :200ms
132 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :300ms
95.6 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :400ms
80.2 ms ± 862 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :500ms
70.7 ms ± 870 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :600ms
64.9 ms ± 591 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :700ms
61.4 ms ± 850 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :800ms
59.2 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :900ms
56.9 ms ± 969 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1000ms
56.8 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1100ms
55.5 ms ± 760 μs per loop (mean ± std. dev. of 7 runs, 

In [ ]:
for i in range(1,4):
    job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
    print("Chunk duration :" + str(500*i) +" ms")
    %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

Chunk duration :500ms
81.1 ms ± 915 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1000ms
85.5 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1500ms
107 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
